In [1]:
#Saturated Oxygen Data Correction
from oxyfloat import OxyFloat, RequiredVariableNotPresent, OpenDAPServerError

of = OxyFloat()

In [2]:
oga_floats = of.get_oxy_floats()

In [5]:
# Testing with a float that has data
oga_floats = ['1900650']
test_file = 'D1900650_137.nc'

In [6]:
for dac_url in of.get_dac_urls(oga_floats):
    for profile_url in of.get_profile_opendap_urls(dac_url):
        if test_file:
            if not profile_url.endswith(test_file):
                continue
        print 'Reading data from ...%s', profile_url[40:]
        try:
            d = of.get_profile_data(profile_url)
        except RequiredVariableNotPresent as e:
            print e
        except OpenDAPServerError as e:
            print e

Reading data from ...%s IOLIS-ARGO-GDAC-OBS/coriolis/1900650/profiles/D1900650_137.nc


In [7]:
# Demonstrate reading the data
print d.keys()
print d['p'][0]['long_name'] + ' (' + d['p'][0]['units'] + ')'
print d['p'][1][:10]
print d['o'][0]['long_name'] + ' (' + d['o'][0]['units'] + ')'
print d['o'][1][:10]

['lon', 'o', 'p', 's', 't', 'lat', 'dt']
SEA PRESSURE (decibar)
[  4.30000019   5.9000001    9.10000038  13.90000057  19.5         24.39999962
  29.5         34.5         39.59999847  44.59999847]
DISSOLVED OXYGEN (micromole/kg)
[ 206.49000549  206.38000488  206.30000305  206.8500061   206.86000061
  211.50999451  218.30999756  232.38000488  216.80000305  190.08000183]


The code below is form the old notebook. Some of it may be folded into oxyfloat functions.

In [ ]:
floatLookup = {}
for url in urls:
    print url
    ds = Dataset(url)
    floatNum = url.split('/')[9]
    print floatNum
    p,t,s,o = (ds.variables['PRES_ADJUSTED'][0][0], ds.variables['TEMP_ADJUSTED'][0][0], 
               ds.variables['PSAL_ADJUSTED'][0][0], ds.variables['DOXY_ADJUSTED'][0][0])
    lat,lon,mtime = (ds.variables['LATITUDE'][0], ds.variables['LONGITUDE'][0], ds.variables['JULD'][0])
    
    # Convert o from micromol/kg to ml/l 
    ##perc_sat = 100 * convertTomll(o) / satO2(s,t) 
    print s,t,perc_sat
    floatLookup[floatNum] = (p,t,s,o,lat,lon,mtime)

In [ ]:
floatLookup

In [33]:
p,t,s,o = (ds.variables['PRES_ADJUSTED'][0][0], ds.variables['TEMP_ADJUSTED'][0][0], ds.variables['PSAL_ADJUSTED'][0][0], ds.variables['DOXY_ADJUSTED'][0][0])

In [53]:
p,t,s,o

(4.3000002, 29.790001, 34.827457, 206.49001)

In [35]:
lat,lon,mtime = (ds.variables['LATITUDE'][0], ds.variables['LONGITUDE'][0], ds.variables['JULD'][0])

In [36]:
lat,lon,mtime

(6.0310000000000006, -14.026, 21985.069212962964)

In [118]:
#The p here is for depth...maybe
#That's why it said the unit was km
def dens_formula(p , t , s):
    C = (999.83 + (5.053*p) - (.048*(p**2))
    B =  .808 - (.0085*p)
    a = .0708*((1 + (.351*p)) + .068*(1 - (.0683*p))*t)
    y = .003*(1 - (.059*p) - .012*(1 - .064*p)*t)
    dens = C + (B*s) - (a*t) - ((y*(35 - s))*t)
    return dens

SyntaxError: invalid syntax (<ipython-input-118-819d3bb21dd0>, line 5)

In [115]:
dens_formula(p , t , s)

1039.220347679988

In [ ]:
def dens_formula(p , t , s):
    p = p / 10.0
    pw = 999.842594 + ((6.793952 * (10**-abs(2)))*t) - ((9.095290 * (10**-abs(3)))*(t**2)) + ((1.001685 * (10**-abs(4)))*(t**3)) - ((1.120083 * (10**-abs(6)))*(t**4)) + ((6.536332 * (10**-abs(9))*(t**5))        
    